In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import h5py
import re

In [2]:
#load csv for 1/10 R2A
R2A_1_10=pd.read_csv("/Users/jennaisrael/julia/dev/DEBmicroTrait.jl-1/files/input/1_10_R2A_medium_with_P.csv")
R2A_1_10.head() #want the "Name" column for plotting

,Name,Quantity,Unit,Compound,Formula,Quantity.1,Unit.1,Molecular_weight,Concentration,Unit.2,Ontology
0,Main Compound,0.05,g,glucose,C6H12O6,100.0,%,180.16,0.277537,mmol/L,Sugars
1,Proteose Peptone,0.05,g,alanine,C3H7NO2,6.0,%,89.09,0.033673,mmol/L,Amino acids
2,Proteose Peptone,0.05,g,arginine,C6H14N4O2,4.7,%,174.20,0.013490,mmol/L,Amino acids
3,Proteose Peptone,0.05,g,aspartic acid,C4H7NO4,0.4,%,133.10,0.001503,mmol/L,Amino acids
4,Proteose Peptone,0.05,g,glutamic acid,C5H9NO4,8.4,%,147.13,0.028546,mmol/L,Amino acids


In [3]:
#basal media recipe, just the NH4Cl, NaH2PO4, KCl, and NaHCO3 in 940 mL not a Liter to account for dilution later
basal_media=pd.read_csv("/Users/jennaisrael/julia/dev/DEBmicroTrait.jl-1/files/input/basal_medium_no_C_recipe_94_percent_volume.csv")
basal_media.head() 

,Name,Quantity,Unit,Compound,Formula,Quantity.1,Unit.1,Molecular_weight,Concentration,Unit.2,Ontology
0,BM,0.25,g,Ammonium Chloride,NH4Cl,100,%,53.4910,4.972004,mmol/L,Nitrogen
1,BM,0.60,g,Sodium Phosphate-monobasic,NaH2PO4,100,%,119.9800,5.320036,mmol/L,Phosphate
2,BM,0.10,g,Potassium Chloride,KCl,100,%,74.5513,1.426977,mmol/L,Potassium
3,BM,2.50,g,Sodium Bicarbonate,NaHCO3,100,%,84.0070,31.658963,mmol/L,Buffer


In [4]:
#amino acid mix 1X
AA=pd.read_csv("/Users/jennaisrael/julia/dev/DEBmicroTrait.jl-1/files/input/Amino_acid_mix_1X.csv")
print(AA) 

                       Name  Quantity Unit                       Compound  \
0         MEM 50X dil to 1X    0.1264    g       L-Arginine hydrochloride   
1         MEM 50X dil to 1X    0.0240    g                      L-Cystine   
2         MEM 50X dil to 1X    0.0420    g  L-Histidine hydrochloride-H2O   
3         MEM 50X dil to 1X    0.0524    g                   L-Isoleucine   
4         MEM 50X dil to 1X    0.0524    g                      L-Leucine   
5         MEM 50X dil to 1X    0.0725    g         L-Lysine hydrochloride   
6         MEM 50X dil to 1X    0.0151    g                   L-Methionine   
7         MEM 50X dil to 1X    0.0330    g                L-Phenylalanine   
8         MEM 50X dil to 1X    0.0476    g                    L-Threonine   
9         MEM 50X dil to 1X    0.0102    g                   L-Tryptophan   
10        MEM 50X dil to 1X    0.0360    g                     L-Tyrosine   
11        MEM 50X dil to 1X    0.0468    g                       L-Valine   

In [5]:
#glucose diluted concentration
glucose=pd.read_csv("/Users/jennaisrael/julia/dev/DEBmicroTrait.jl-1/files/input/basal_medium.csv")
#old file need to update concentration
glucose.Quantity=0.207
glucose.Concentration=0.207/glucose.Molecular_weight *1000
glucose.head() 


,Name,Quantity,Unit,Compound,Formula,Quantity.1,Unit.1,Molecular_weight,Concentration,Unit.2,Ontology
0,Main Compound,0.207,g,glucose,C6H12O6,100,%,180.16,1.148979,mmol/L,Sugars


In [6]:
#lysine and glycine
lys_gly=AA.loc[(AA.Compound=='L-Lysine hydrochloride') | (AA.Compound=='Glycine'),:].reset_index() #OR logical indexing
#reset indices
lys_gly['Name']=['Lysine','Glycine']
lys_gly['Quantity']=[0.0005,0.0005] #equivalent to 10^-7 g/ 200 uL which is about an order more than they need
lys_gly['Compound'][0]='Lysine'
lys_gly['Formula'][0]='C6H14N2O2'
lys_gly['Molecular_weight'][0]=146.19
lys_gly['Concentration']=lys_gly['Quantity'].div(lys_gly['Molecular_weight'])*1000
lys_gly.head()

/var/folders/2j/j0k91wbd7lj88qswp8htny9m0000gn/T/ipykernel_90568/3798845327.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lys_gly['Compound'][0]='Lysine'
/var/folders/2j/j0k91wbd7lj88qswp8htny9m0000gn/T/ipykernel_90568/3798845327.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lys_gly['Formula'][0]='C6H14N2O2'
/var/folders/2j/j0k91wbd7lj88qswp8htny9m0000gn/T/ipykernel_90568/3798845327.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

,index,Name,Quantity,Unit,Compound,Formula,Quantity.1,Unit.1,Molecular_weight,Concentration,Unit.2,Ontology
0,5,Lysine,0.0005,g,Lysine,C6H14N2O2,100,%,146.19,0.003420,mmol/L,Amino acids
1,12,Glycine,0.0005,g,Glycine,C2H5NO2,100,%,75.00,0.006667,mmol/L,Amino acids


In [7]:
print(type(lys_gly))

<class 'pandas.core.frame.DataFrame'>


In [8]:
#print(re.findall(r"(\d+)M", s))
s='C6H12'
print(type(re.findall(r"C(\d+)", s)))
print(float(re.findall(r"C(\d+)", s)[0]))

<class 'list'>
6.0


In [9]:

print(float(re.findall(r"C(\d+)",R2A_1_10['Formula'][0])[0]))

6.0


In [10]:
print(re.search(r"N(\w+)",'CNO'))

<re.Match object; span=(1, 3), match='NO'>


In [11]:
for f in lys_gly['Formula']:
    if re.search(r"P(\d+)",f):
        print(float(re.findall(r"C(\d+)",f)[0]))
    elif re.search(r"P(\w+)",f): #if the letter is there but doesn't have number after it assume stoichiometry 1
        print(1)
    else:
       print(0)

0
0


In [12]:
f

'C2H5NO2'

In [13]:
print(re.search(r"C(\d+)",'C6'))

<re.Match object; span=(0, 2), match='C6'>


In [14]:
#convert to C,N,P stoichiometry
def extract_CNP(df):
# extract the total concentration of C, N, and P from each file
#extract stoichiometry from 'Formula' string
    C_stoic=[]
    for f in df['Formula']:
        if re.search(r"C(\d+)",f):
            C_stoic.append(float(re.findall(r"C(\d+)",f)[0]))
        elif re.search(r"C(\w+)",f):
            C_stoic.append(float(1))
        else:
            C_stoic.append(float(0))

    #add a column for C stoichiometry
    df['C_stoic']=C_stoic

    #add a column for concentration of C in mmol/L
    df['C_conc']=df['C_stoic'].multiply(df['Concentration'])
    df.head()

    #repeat for N and P
    N_stoic=[]
    for f in df['Formula']:
        if re.search(r"N(\d+)",f):
            N_stoic.append(float(re.findall(r"N(\d+)",f)[0]))
        elif re.search(r"N(\w+)",f):
            N_stoic.append(float(1))
        else:
            N_stoic.append(float(0))

    #add a column for C stoichiometry
    df['N_stoic']=N_stoic

    #add a column for concentration of C in mmol/L
    df['N_conc']=df['N_stoic'].multiply(df['Concentration'])


    P_stoic=[]
    for f in df['Formula']:
        if re.search(r"P(\d+)",f):
            P_stoic.append(float(re.findall(r"P(\d+)",f)[0]))
        elif re.search(r"P(\w+)",f):
            P_stoic.append(float(1))
        else:
            P_stoic.append(float(0))

    #add a column for C stoichiometry
    df['P_stoic']=P_stoic

    #add a column for concentration of C in mmol/L
    df['P_conc']=df['P_stoic'].multiply(df['Concentration'])

    df_new= df
    #output the updated dataframe AND a vector of total C,N,P concentrations
    CNP=[df['C_conc'].sum(), df['N_conc'].sum(), df['P_conc'].sum()]

    #return df_new, CNP 
    return CNP 
#return the updated data frame and the vector of CNP concentrations

In [15]:
# [df_new, CNP] = extract_CNP(lys_gly)
# df_new.tail()

print(extract_CNP(lys_gly))


[0.033854572816198095, 0.013507079827621588, 0.0]


In [16]:
#load media recipes
names=['1/10_R2A','BM+G','BM+G+V+M','BM+G+V+M+1XAA','BM+G+V+M+0.1XAA','BM+G+V+M+0.01XAA','BM+G+gly+lys','BM+G+gly+lys+V+M'] #use these as row indices
ingredients=['1/10_R2A','BM','G','1XAA','lys_gly']
elements=['C','N','P']
#these vectors multiply the concentrations in each of the media dataframes
vol_R2A=np.array([180,0,0,0,0,0,0,0])*(1/200)
vol_BM=np.array([20.0,198.0,194.0,198.0,193.4,193.94,194.0,190.0])*(1/200) #volume of basal media in uL out of 200 uL, including cells and diluted amino acids
vol_glu=[0,1,1,1,1,1,1,1] #spreadsheet already diluted
vol_AA=[0,0,0,1,0.1,0.01,0,0] #spreadsheet alredy diluted to 1x
vol_lys_gly=[0,0,0,0,0,0,1,1]#spreadsheet already diluted
media_recipes=pd.DataFrame(columns=names, index=ingredients,data=[vol_R2A,vol_BM,vol_glu,vol_AA,vol_lys_gly])
media_recipes.head()

,1/10_R2A,BM+G,BM+G+V+M,BM+G+V+M+1XAA,BM+G+V+M+0.1XAA,BM+G+V+M+0.01XAA,BM+G+gly+lys,BM+G+gly+lys+V+M
1/10_R2A,0.9,0.00,0.00,0.00,0.000,0.0000,0.00,0.00
BM,0.1,0.99,0.97,0.99,0.967,0.9697,0.97,0.95
G,0.0,1.00,1.00,1.00,1.000,1.0000,1.00,1.00
1XAA,0.0,0.00,0.00,1.00,0.100,0.0100,0.00,0.00
lys_gly,0.0,0.00,0.00,0.00,0.000,0.0000,1.00,1.00


In [17]:
#transpose this dataframe so we can multiply to get desired output
media_recipes_transpose=media_recipes.T
media_recipes_transpose.head()

,1/10_R2A,BM,G,1XAA,lys_gly
1/10_R2A,0.9,0.100,0.0,0.0,0.0
BM+G,0.0,0.990,1.0,0.0,0.0
BM+G+V+M,0.0,0.970,1.0,0.0,0.0
BM+G+V+M+1XAA,0.0,0.990,1.0,1.0,0.0
BM+G+V+M+0.1XAA,0.0,0.967,1.0,0.1,0.0


In [18]:
#create a dataframe of C,N,P concentrations from the media mixes
#media_CNP=pd.DataFrame(columns=names,index=elements)
# first call the homemade extract_CNP(df) function on each of the media types

CNP_R2A_1_10 = extract_CNP(R2A_1_10)
#print(CNP_R2A_1_10)
CNP_BM = extract_CNP(basal_media)
CNP_BM[0]=float(0) #currently function is counting bicarbonate as C so leave that out for now
#print(CNP_BM)
CNP_G = extract_CNP(glucose)
#print(CNP_G)
CNP_AA = extract_CNP(AA)
#print(CNP_AA)
CNP_lys_gly = extract_CNP(lys_gly)
#print(lys_gly)



CNP_of_ing=pd.DataFrame(columns=elements,index=ingredients,data=[CNP_R2A_1_10, CNP_BM, CNP_G, CNP_AA, CNP_lys_gly])
CNP_of_ing.head()




# # # loop over each media type to multiply weights from media_recipes dataframe by the CNP vector of each ingredient
# for m in media_recipes.columns():
    
# # sum over each element and add to media_CNP dataframe 
#     media_CNP[m]=


,C,N,P
1/10_R2A,6.905141,0.883018,0.036485
BM,0.000000,41.951002,5.320036
G,6.893872,0.000000,0.000000
1XAA,23.519471,6.192621,0.000000
lys_gly,0.033855,0.013507,0.000000


In [19]:
media_CNP=media_recipes_transpose.dot(CNP_of_ing)
media_CNP

,C,N,P
1/10_R2A,6.214627,4.989817,0.564840
BM+G,6.893872,41.531492,5.266835
BM+G+V+M,6.893872,40.692472,5.160435
BM+G+V+M+1XAA,30.413343,47.724112,5.266835
BM+G+V+M+0.1XAA,9.245819,41.185881,5.144474
BM+G+V+M+0.01XAA,7.129067,40.741813,5.158839
BM+G+gly+lys,6.927727,40.705979,5.160435
BM+G+gly+lys+V+M,6.927727,39.866959,5.054034


In [20]:
media_CNP['CN_ratio']=media_CNP['C'].div(media_CNP['N'])
media_CNP

,C,N,P,CN_ratio
1/10_R2A,6.214627,4.989817,0.564840,1.245462
BM+G,6.893872,41.531492,5.266835,0.165991
BM+G+V+M,6.893872,40.692472,5.160435,0.169414
BM+G+V+M+1XAA,30.413343,47.724112,5.266835,0.637274
BM+G+V+M+0.1XAA,9.245819,41.185881,5.144474,0.224490
BM+G+V+M+0.01XAA,7.129067,40.741813,5.158839,0.174982
BM+G+gly+lys,6.927727,40.705979,5.160435,0.170189
BM+G+gly+lys+V+M,6.927727,39.866959,5.054034,0.173771


In [21]:
# #how much glycine and lysine are in each of these media types?
# media_lg_content=pd.DataFrame(columns=['Lysine','Glycine'],index=names) # what we want in the end
ing_lg_content=pd.DataFrame(columns=['Lysine','Glycine'],index=ingredients,data=np.zeros([5,2])) # what we need to multiply by media recipes by (create this first )

#sum over multiple entries in R2A
#pull_LG=R2A_1_10[(R2A_1_10['Compound']== 'glycine') | (R2A_1_10['Compound']== 'lysine')]
ing_lg_content.loc['1/10_R2A',:]=[R2A_1_10[R2A_1_10['Compound']== 'glycine']['Concentration'].sum(), R2A_1_10[R2A_1_10['Compound']== 'lysine']['Concentration'].sum()]

#Amino acid dilutions
ing_lg_content.loc['1XAA',:]=[AA[AA.Compound=='L-Lysine hydrochloride']['Concentration'], AA[AA.Compound=='Glycine']['Concentration']]

#lysine and glycine
ing_lg_content.loc['lys_gly',:]=[lys_gly[lys_gly.Compound=='Lysine']['Concentration'], lys_gly[lys_gly.Compound=='Glycine']['Concentration']]
ing_lg_content

,Lysine,Glycine
1/10_R2A,0.083925,0.047883
BM,0.000000,0.000000
G,0.000000,0.000000
1XAA,0.396175,0.100000
lys_gly,0.003420,0.006667


In [22]:
#multiply the recipes by the lysine and glycine in each ingredient
media_LG=media_recipes_transpose.dot(ing_lg_content)
media_LG

,Lysine,Glycine
1/10_R2A,0.075533,0.043095
BM+G,0.000000,0.000000
BM+G+V+M,0.000000,0.000000
BM+G+V+M+1XAA,0.396175,0.100000
BM+G+V+M+0.1XAA,0.039617,0.010000
BM+G+V+M+0.01XAA,0.003962,0.001000
BM+G+gly+lys,0.003420,0.006667
BM+G+gly+lys+V+M,0.003420,0.006667


##REPEAT FOR MEDIA USED IN SECOND EXPERIMENT

In [23]:
#need new recipe dataframe then update lysine and glycine conentrations and multiply by content of each component as before

In [24]:
lys_gly_2=lys_gly.copy()
lys_gly_2['Quantity']=[0.01226,0.00359] #concentration in a L once diluted
lys_gly_2['Concentration']=lys_gly_2['Quantity'].div(lys_gly_2['Molecular_weight'])*1000
lys_gly_2

,index,Name,Quantity,Unit,Compound,Formula,Quantity.1,Unit.1,Molecular_weight,Concentration,Unit.2,Ontology,C_stoic,C_conc,N_stoic,N_conc,P_stoic,P_conc
0,5,Lysine,0.01226,g,Lysine,C6H14N2O2,100,%,146.19,0.083863,mmol/L,Amino acids,6.0,0.020521,2.0,0.006840,0.0,0.0
1,12,Glycine,0.00359,g,Glycine,C2H5NO2,100,%,75.00,0.047867,mmol/L,Amino acids,2.0,0.013333,1.0,0.006667,0.0,0.0


In [25]:
new_q=np.array(AA.loc[(AA.Name=='MEM 50X dil to 1X'),:]['Quantity']*0.5)
new_c=np.array(AA.loc[(AA.Name=='MEM 50X dil to 1X'),:]['Concentration']*0.5)
type(new_c)

numpy.ndarray

In [26]:
#Accidentally added half the 5X AA this time as last time (0.5X not 1X for some of the amino acids)
AA_2=AA.copy()
new_q=np.array(AA.loc[(AA.Name=='MEM 50X dil to 1X'),:]['Quantity']*0.5)
new_c=np.array(AA.loc[(AA.Name=='MEM 50X dil to 1X'),:]['Concentration']*0.5)
AA_2.loc[(AA_2.Name=='MEM 50X dil to 1X'),'Quantity']=new_q
AA_2.loc[(AA_2.Name=='MEM 50X dil to 1X'),'Concentration']=new_c
AA_2

,Name,Quantity,Unit,Compound,Formula,Quantity.1,Unit.1,Molecular_weight,Concentration,Unit.2,Ontology,C_stoic,C_conc,N_stoic,N_conc,P_stoic,P_conc
0,MEM 50X dil to 1X,0.06320,g,L-Arginine hydrochloride,C6H14N4O2,100,%,211,0.299526,mmol/L,Amino acids,6.0,3.594313,4.0,2.396208,0.0,0.0
1,MEM 50X dil to 1X,0.01200,g,L-Cystine,C6H12N2O4S2,100,%,240,0.050000,mmol/L,Amino acids,6.0,0.600000,2.0,0.200000,0.0,0.0
2,MEM 50X dil to 1X,0.02100,g,L-Histidine hydrochloride-H2O,C6H9NO2,100,%,210,0.100000,mmol/L,Amino acids,6.0,1.200000,1.0,0.200000,0.0,0.0
3,MEM 50X dil to 1X,0.02620,g,L-Isoleucine,C6H13NO2,100,%,131,0.200000,mmol/L,Amino acids,6.0,2.400000,1.0,0.400000,0.0,0.0
4,MEM 50X dil to 1X,0.02620,g,L-Leucine,C6H13NO3,100,%,131,0.200000,mmol/L,Amino acids,6.0,2.400000,1.0,0.400000,0.0,0.0
5,MEM 50X dil to 1X,0.03625,g,L-Lysine hydrochloride,C6H13NO3,100,%,183,0.198087,mmol/L,Amino acids,6.0,2.377049,1.0,0.396175,0.0,0.0
6,MEM 50X dil to 1X,0.00755,g,L-Methionine,C6H13NO3,100,%,149,0.050671,mmol/L,Amino acids,6.0,0.608054,1.0,0.101342,0.0,0.0
7,MEM 50X dil to 1X,0.01650,g,L-Phenylalanine,C9H11NO2,100,%,165,0.100000,mmol/L,Amino acids,9.0,1.800000,1.0,0.200000,0.0,0.0
8,MEM 50X dil to 1X,0.02380,g,L-Threonine,C4H9NO3,100,%,119,0.200000,mmol/L,Amino acids,4.0,1.600000,1.0,0.400000,0.0,0.0
9,MEM 50X dil to 1X,0.00510,g,L-Tryptophan,C11H12N2O2,100,%,204,0.025000,mmol/L,Amino acids,11.0,0.550000,2.0,0.100000,0.0,0.0


In [27]:
CNP_lys_gly_2 = extract_CNP(lys_gly_2)
CNP_AA_2=extract_CNP(AA_2)
CNP_of_ing_2=pd.DataFrame(columns=elements,index=ingredients,data=[CNP_R2A_1_10, CNP_BM, CNP_G, CNP_AA_2, CNP_lys_gly_2])
CNP_of_ing_2.head()

,C,N,P
1/10_R2A,6.905141,0.883018,0.036485
BM,0.000000,41.951002,5.320036
G,6.893872,0.000000,0.000000
1XAA,13.059735,3.496310,0.000000
lys_gly,0.598914,0.215594,0.000000


In [28]:
#New media recipes
names=['1/10_R2A','BM+G+new(gly+lys)','BM+G+new(gly+lys)+V+M','BM+G','BM+G+0.1XAA','BM+G+V+M+0.1XAA+V+M'] #use these as row indices
ingredients=['1/10_R2A','BM','G','1XAA','lys_gly']
elements=['C','N','P']
#these vectors multiply the concentrations in each of the media dataframes
vol_R2A=np.array([180,0,0,0,0,0])*(1/200)
vol_BM=np.array([20.0,196.0,192.0,198.0,199.6,195.6])*(1/202) #volume of basal media in uL out of 200 uL, including cells and diluted amino acids
vol_glu=[0,1,1,1,1,1] #spreadsheet already diluted
vol_AA=[0,0,0,0,0.1,0.1] #spreadsheet alredy diluted to 1x
vol_lys_gly=[0,1,1,0,0,0]#spreadsheet already diluted
media_recipes_2=pd.DataFrame(columns=names, index=ingredients,data=[vol_R2A,vol_BM,vol_glu,vol_AA,vol_lys_gly])
media_recipes_2.head()

,1/10_R2A,BM+G+new(gly+lys),BM+G+new(gly+lys)+V+M,BM+G,BM+G+0.1XAA,BM+G+V+M+0.1XAA+V+M
1/10_R2A,0.90000,0.000000,0.000000,0.000000,0.000000,0.000000
BM,0.09901,0.970297,0.950495,0.980198,0.988119,0.968317
G,0.00000,1.000000,1.000000,1.000000,1.000000,1.000000
1XAA,0.00000,0.000000,0.000000,0.000000,0.100000,0.100000
lys_gly,0.00000,1.000000,1.000000,0.000000,0.000000,0.000000


In [29]:
media_recipes_2_transpose=media_recipes_2.T
media_recipes_2_transpose

,1/10_R2A,BM,G,1XAA,lys_gly
1/10_R2A,0.9,0.099010,0.0,0.0,0.0
BM+G+new(gly+lys),0.0,0.970297,1.0,0.0,1.0
BM+G+new(gly+lys)+V+M,0.0,0.950495,1.0,0.0,1.0
BM+G,0.0,0.980198,1.0,0.0,0.0
BM+G+0.1XAA,0.0,0.988119,1.0,0.1,0.0
BM+G+V+M+0.1XAA+V+M,0.0,0.968317,1.0,0.1,0.0


In [30]:
media_CNP_2=media_recipes_2_transpose.dot(CNP_of_ing_2)
media_CNP_2

,C,N,P
1/10_R2A,6.214627,4.948281,0.559573
BM+G+new(gly+lys),7.492786,40.920526,5.162015
BM+G+new(gly+lys)+V+M,7.492786,40.089813,5.056668
BM+G,6.893872,41.120289,5.214688
BM+G+0.1XAA,8.199846,41.802205,5.256827
BM+G+V+M+0.1XAA+V+M,8.199846,40.971492,5.151480


In [31]:
media_CNP_2['CN_ratio']=media_CNP_2['C'].div(media_CNP_2['N'])
media_CNP_2

,C,N,P,CN_ratio
1/10_R2A,6.214627,4.948281,0.559573,1.255916
BM+G+new(gly+lys),7.492786,40.920526,5.162015,0.183106
BM+G+new(gly+lys)+V+M,7.492786,40.089813,5.056668,0.186900
BM+G,6.893872,41.120289,5.214688,0.167651
BM+G+0.1XAA,8.199846,41.802205,5.256827,0.196158
BM+G+V+M+0.1XAA+V+M,8.199846,40.971492,5.151480,0.200135


In [32]:
# #how much glycine and lysine are in each of these media types?
# media_lg_content=pd.DataFrame(columns=['Lysine','Glycine'],index=names) # what we want in the end
ing_lg_content_2=pd.DataFrame(columns=['Lysine','Glycine'],index=ingredients,data=np.zeros([5,2])) # what we need to multiply by media recipes by (create this first )

#sum over multiple entries in R2A
#pull_LG=R2A_1_10[(R2A_1_10['Compound']== 'glycine') | (R2A_1_10['Compound']== 'lysine')]
ing_lg_content_2.loc['1/10_R2A',:]=[R2A_1_10[R2A_1_10['Compound']== 'glycine']['Concentration'].sum(), R2A_1_10[R2A_1_10['Compound']== 'lysine']['Concentration'].sum()]

#Amino acid dilutions
ing_lg_content_2.loc['1XAA',:]=[AA_2[AA_2.Compound=='L-Lysine hydrochloride']['Concentration'], AA_2[AA_2.Compound=='Glycine']['Concentration']]

#lysine and glycine
ing_lg_content_2.loc['lys_gly',:]=[lys_gly_2[lys_gly_2.Compound=='Lysine']['Concentration'], lys_gly_2[lys_gly_2.Compound=='Glycine']['Concentration']]
ing_lg_content_2

,Lysine,Glycine
1/10_R2A,0.083925,0.047883
BM,0.000000,0.000000
G,0.000000,0.000000
1XAA,0.198087,0.100000
lys_gly,0.083863,0.047867


In [33]:
#multiply the recipes by the lysine and glycine in each ingredient
media_LG_2=media_recipes_2_transpose.dot(ing_lg_content_2)
media_LG_2

,Lysine,Glycine
1/10_R2A,0.075533,0.043095
BM+G+new(gly+lys),0.083863,0.047867
BM+G+new(gly+lys)+V+M,0.083863,0.047867
BM+G,0.000000,0.000000
BM+G+0.1XAA,0.019809,0.010000
BM+G+V+M+0.1XAA+V+M,0.019809,0.010000


## Try increasing C concentration to make sure there are sufficient vitamins and minerals

In [34]:
# #need a new glucose dataframe
# #old file need to update concentration
# glucose2=glucose
# glucose2.Concentration=4.97 #mM
# glucose2.Quantity=glucose.Molecular_weight *glucose2.Concentration* 10^(-3)
# glucose2.head()

In [35]:
4.97/1.15

4.321739130434783

In [36]:
#New media recipes
names=['1/10_R2A','BM+1.15G+V+M+1XAA+V+M','BM+4.97G+V+M+1XAA+V+M', 'BM+1.15G+V+M+0.1XAA+V+M','BM+4.97G+V+M+0.1XAA+V+M','BM+G+2Xnew(gly+lys)+V+M'] #use these as row indices
ingredients=['1/10_R2A','BM','G','1XAA','lys_gly']
elements=['C','N','P']
#these vectors multiply the concentrations in each of the media dataframes
vol_R2A=np.array([180,0,0,0,0,0])*(1/200)
vol_BM=np.array([20.0,188.0,188.0,188.0,188.0,190.0])*(1/200) #volume of basal media in uL out of 200 uL, including cells and diluted amino acids
vol_glu=[0,1,4.32,1,4.32,1] #spreadsheet already diluted
vol_AA=[0,1,1,0.1,0.1,0] #spreadsheet alredy diluted to 1x
vol_lys_gly=[0,0,0,0,0,1]#spreadsheet already diluted
media_recipes_3=pd.DataFrame(columns=names, index=ingredients,data=[vol_R2A,vol_BM,vol_glu,vol_AA,vol_lys_gly])
media_recipes_3.head()

,1/10_R2A,BM+1.15G+V+M+1XAA+V+M,BM+4.97G+V+M+1XAA+V+M,BM+1.15G+V+M+0.1XAA+V+M,BM+4.97G+V+M+0.1XAA+V+M,BM+G+2Xnew(gly+lys)+V+M
1/10_R2A,0.9,0.00,0.00,0.00,0.00,0.00
BM,0.1,0.94,0.94,0.94,0.94,0.95
G,0.0,1.00,4.32,1.00,4.32,1.00
1XAA,0.0,1.00,1.00,0.10,0.10,0.00
lys_gly,0.0,0.00,0.00,0.00,0.00,1.00


In [37]:
media_recipes_3_transpose=media_recipes_3.T
media_recipes_3_transpose

,1/10_R2A,BM,G,1XAA,lys_gly
1/10_R2A,0.9,0.10,0.00,0.0,0.0
BM+1.15G+V+M+1XAA+V+M,0.0,0.94,1.00,1.0,0.0
BM+4.97G+V+M+1XAA+V+M,0.0,0.94,4.32,1.0,0.0
BM+1.15G+V+M+0.1XAA+V+M,0.0,0.94,1.00,0.1,0.0
BM+4.97G+V+M+0.1XAA+V+M,0.0,0.94,4.32,0.1,0.0
BM+G+2Xnew(gly+lys)+V+M,0.0,0.95,1.00,0.0,1.0


In [38]:
#edit CNP of ing to reflect full concentration of 50X AA this time (switch to CNP_AA)
CNP_of_ing_3=pd.DataFrame(columns=elements,index=ingredients,data=[CNP_R2A_1_10, CNP_BM, CNP_G, CNP_AA, CNP_lys_gly_2])
CNP_of_ing_3.head()

,C,N,P
1/10_R2A,6.905141,0.883018,0.036485
BM,0.000000,41.951002,5.320036
G,6.893872,0.000000,0.000000
1XAA,23.519471,6.192621,0.000000
lys_gly,0.598914,0.215594,0.000000


In [39]:
media_CNP_3=media_recipes_3_transpose.dot(CNP_of_ing_3)
media_CNP_3

,C,N,P
1/10_R2A,6.214627,4.989817,0.564840
BM+1.15G+V+M+1XAA+V+M,30.413343,45.626562,5.000833
BM+4.97G+V+M+1XAA+V+M,53.300998,45.626562,5.000833
BM+1.15G+V+M+0.1XAA+V+M,9.245819,40.053204,5.000833
BM+4.97G+V+M+0.1XAA+V+M,32.133475,40.053204,5.000833
BM+G+2Xnew(gly+lys)+V+M,7.492786,40.069045,5.054034


In [40]:
media_LG_3=media_recipes_3_transpose.dot(ing_lg_content_2)
media_LG_3

,Lysine,Glycine
1/10_R2A,0.075533,0.043095
BM+1.15G+V+M+1XAA+V+M,0.198087,0.100000
BM+4.97G+V+M+1XAA+V+M,0.198087,0.100000
BM+1.15G+V+M+0.1XAA+V+M,0.019809,0.010000
BM+4.97G+V+M+0.1XAA+V+M,0.019809,0.010000
BM+G+2Xnew(gly+lys)+V+M,0.083863,0.047867
